In [1]:
import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2)))
        for n in range(random.randint(N, N * 5))
    ]

N = 100
POPULATION_SIZE = 2000
# N_PARTECIPANTS = 5
MAX_USELESS_EPOCHS = 1000
SETS = problem(N, seed=42)

In [2]:
def fitness_uncovered_elements(current_set, N):
    s = set()
    for i in range(len(current_set)):
        if current_set[i] == 1: s.update(SETS[i])
    return N - len(s)

def fitness_weight_sets(current_set):
    tot = 0
    for i in range(len(current_set)):
        if current_set[i] == 1: tot += len(SETS[i])
    return tot

def crossover(mom, dad, mode=2):
    if mode == 1:
        return one_point_crossover(mom, dad), one_point_crossover(dad, mom)
    else:
        return two_point_crossover(mom, dad), two_point_crossover(dad, mom)

def one_point_crossover(mom, dad):
    # randomly select a crossover point
    crossover_point = random.randint(0, len(mom) - 1)
    # create a new individual by combining the first half of mom with the second half of dad
    return mom[:crossover_point] + dad[crossover_point:]

def two_point_crossover(mom, dad):
    # randomly select a crossover point
    crossover_point_1 = random.randint(0, len(mom) - 1)
    crossover_point_2 = random.randint(0, len(mom) - 1)
    while crossover_point_2 == crossover_point_1:
        crossover_point_2 = random.randint(0, len(mom) - 1)
    
    # create a new individual by combining the first and thirds half of mom with the middle half of dad
    lower_cut = min(crossover_point_1,crossover_point_2)
    upper_cut = max(crossover_point_1,crossover_point_2)
    return mom[:lower_cut] + dad[lower_cut:upper_cut] + mom[upper_cut:]

# not actually used, I used this for some testing
def tournament(population, n_partecipants):
    return max(random.sample(population, n_partecipants), key=lambda x: (x[1], x[2]))

def mate(population, N):
    useless_epochs = 0
    best = None
    while useless_epochs < MAX_USELESS_EPOCHS:
        if random.random() < 0.8:
            #crossover
            for i in range(len(population)):
                # first mate the 2 fittest indivudals (with elitism, didn't really work)
                # mom = mating_population[0 if i == 0 else random.randint(2, len(mating_population) - 1)][0]
                # dad = mating_population[1 if i == 0 else random.randint(2, len(mating_population) - 1)][0]

                # first mate the 2 fittest indivudals (without elitism, better results)
                mom = population[random.randint(0, len(population) - 1)][0]
                dad = population[random.randint(0, len(population) - 1)][0]
                offspring_1, offspring_2 = crossover(mom, dad, mode=2)
                population.append((offspring_1, fitness_uncovered_elements(offspring_1, N), fitness_weight_sets(offspring_1)))
                population.append((offspring_2, fitness_uncovered_elements(offspring_2, N), fitness_weight_sets(offspring_2)))
        else:
            # mutation
            for p in population:
                if random.random() < 0.1:
                    for _ in range(random.randrange(5)):
                        index = random.randint(0, len(p) - 1)
                        mutated_list = p[0][:]
                        mutated_list[index] = 1 if mutated_list[index] == 0 else 0
                        p = (mutated_list, fitness_uncovered_elements(mutated_list, N), fitness_weight_sets(mutated_list))

        # cleaning
        population = sorted(population, key=lambda x: (x[1], x[2]))[:POPULATION_SIZE]

        # first one is the best
        if best is None or (best[2] > population[0][2] and population[0][1] == 0):
            best = population[0]
            useless_epochs = 0
        else:
            useless_epochs += 1

    return best

def check_solution(solution, N):
    s = set()
    for i in range(len(solution)):
        if solution[i] == 1: s.update(SETS[i])
    return len(s) == N

In [3]:
# rappresentation: [[bynary string]]
binrappr = [[random.choice([0,1]) for _ in SETS] for _ in range(POPULATION_SIZE)]
# rappresentation: [(binary_rappr, fit_uncovered_elements, len_used_subsets)]
fitvalues = sorted([(p, fitness_uncovered_elements(p, N), fitness_weight_sets(p)) for p in binrappr], key=lambda x: (x[1], x[2]))

# start di algorithm
best = mate(fitvalues, N)

# print results
print(f"Best for {N} elements\nStats: {best[1]} uncovered elements\nWeight: {best[2]}\nSolution check: {check_solution(best[0], N)}")
print([SETS[s] for s in range(len(best[0])) if best[0][s] == 1])


Best for 100 elements
Stats: 0 uncovered elements
Weight: 192
Solution check: True
[{4, 5, 12, 14, 17, 19, 27, 30, 35, 38, 47, 49, 52, 53, 56, 58, 59, 62, 63, 68, 69, 75, 78, 83, 85, 88, 89, 93, 95}, {0, 1, 2, 3, 6, 8, 14, 16, 18, 22, 30, 32, 33, 35, 41, 45, 53, 67, 69, 75, 76, 84, 94, 95, 99}, {6, 9, 12, 15, 16, 18, 24, 25, 35, 36, 43, 46, 47, 49, 50, 51, 57, 64, 65, 70, 71, 72, 77, 78, 79, 87, 96, 97}, {97, 34, 65, 3, 70, 39, 71, 11, 13, 46, 45, 16, 52, 55, 23, 56, 57}, {0, 1, 2, 13, 16, 22, 26, 27, 28, 32, 34, 35, 38, 39, 44, 48, 50, 52, 54, 58, 59, 60, 65, 66, 73, 74, 76, 77, 79, 82, 83, 86, 87, 90, 91, 98, 99}, {2, 5, 7, 14, 15, 16, 17, 18, 20, 21, 23, 26, 29, 31, 33, 42, 51, 52, 53, 61, 65, 66, 72, 73, 75, 76, 80, 81, 82, 84, 86, 91, 92}, {3, 5, 10, 29, 34, 37, 40, 43, 45, 47, 48, 52, 54, 55, 56, 57, 59, 63, 65, 66, 74, 79, 96}]
